In [1]:
import geopy
import geopandas as gpd
import pandas as pd
from geopy.geocoders import Nominatim
from geopy.extra.rate_limiter import RateLimiter
from halo import Halo
import folium
from folium.map import *
from folium.plugins import MiniMap, Search, BeautifyIcon
from folium.features import GeoJson, GeoJsonTooltip, CustomIcon
import time
import json
import requests
import branca

In [2]:
data_url = 'https://raw.githubusercontent.com/owid/covid-19-data/master/public/data'
data_request = f'{data_url}/owid-covid-data.csv'
data = pd.read_csv(data_request,usecols=['location','total_cases','total_deaths'])

In [3]:
data

,location,total_cases,total_deaths
0,Aruba,2,0
1,Aruba,4,0
2,Aruba,12,0
3,Aruba,17,0
4,Aruba,19,0
...,...,...,...
14497,International,705,4
14498,International,705,6
14499,International,705,6
14500,International,705,6


In [4]:
grouped_data = data.groupby('location').max().reset_index()

In [5]:
grouped_data

,location,total_cases,total_deaths
0,Afghanistan,2171,64
1,Albania,773,31
2,Algeria,4006,450
3,Andorra,745,42
4,Angola,27,2
...,...,...,...
205,Western Sahara,6,0
206,World,3214256,232570
207,Yemen,6,2
208,Zambia,106,3


In [6]:
name_list = []
cases_list = []
deaths_list = []
for row in grouped_data.values:
    if row[0] == 'World' or row[0] == 'International':
        pass
    elif row[0] == 'United States':
        name = row[0].replace('United States','United States of America')
        name_list.append(name)
        cases_list.append(row[1])
        deaths_list.append(row[2])
        
    elif row[0] == 'Democratic Republic of Congo':
        name = row[0].replace('Democratic Republic of Congo','Democratic Republic of the Congo')
        name_list.append(name)
        cases_list.append(row[1])
        deaths_list.append(row[2])
    elif row[0] == 'Tanzania':
        name = row[0].replace('Tanzania','United Republic of Tanzania')
        name_list.append(name)
        cases_list.append(row[1])
        deaths_list.append(row[2])
    elif row[0] == 'Congo':
        name = row[0].replace('Congo','Republic of the Congo')
        name_list.append(name)
        cases_list.append(row[1])
        deaths_list.append(row[2])
    else:
        name = row[0]
        name_list.append(name)
        cases_list.append(row[1])
        deaths_list.append(row[2])



In [7]:
new_data = pd.DataFrame(list(zip(name_list,cases_list,deaths_list)), columns=['name','cases','deaths'])
new_data

,name,cases,deaths
0,Afghanistan,2171,64
1,Albania,773,31
2,Algeria,4006,450
3,Andorra,745,42
4,Angola,27,2
...,...,...,...
203,Vietnam,270,0
204,Western Sahara,6,0
205,Yemen,6,2
206,Zambia,106,3


In [8]:
geo_url = 'https://raw.githubusercontent.com/python-visualization/folium/master/examples/data'
geo_data = f'{geo_url}/world-countries.json'
geo_json_data = json.loads(requests.get(geo_data).text)

In [9]:
geo_json_data

{'type': 'FeatureCollection',
 'features': [{'type': 'Feature',
   'properties': {'name': 'Afghanistan'},
   'geometry': {'type': 'Polygon',
    'coordinates': [[[61.210817, 35.650072],
      [62.230651, 35.270664],
      [62.984662, 35.404041],
      [63.193538, 35.857166],
      [63.982896, 36.007957],
      [64.546479, 36.312073],
      [64.746105, 37.111818],
      [65.588948, 37.305217],
      [65.745631, 37.661164],
      [66.217385, 37.39379],
      [66.518607, 37.362784],
      [67.075782, 37.356144],
      [67.83, 37.144994],
      [68.135562, 37.023115],
      [68.859446, 37.344336],
      [69.196273, 37.151144],
      [69.518785, 37.608997],
      [70.116578, 37.588223],
      [70.270574, 37.735165],
      [70.376304, 38.138396],
      [70.806821, 38.486282],
      [71.348131, 38.258905],
      [71.239404, 37.953265],
      [71.541918, 37.905774],
      [71.448693, 37.065645],
      [71.844638, 36.738171],
      [72.193041, 36.948288],
      [72.63689, 37.047558],
      [73.

In [10]:
states = gpd.GeoDataFrame.from_features(geo_json_data,crs='EPSG:4326')
states.head()

,geometry,name
0,"POLYGON ((61.21082 35.65007, 62.23065 35.27066...",Afghanistan
1,"MULTIPOLYGON (((16.32653 -5.87747, 16.57318 -6...",Angola
2,"POLYGON ((20.59025 41.85540, 20.46317 41.51509...",Albania
3,"POLYGON ((51.57952 24.24550, 51.75744 24.29407...",United Arab Emirates
4,"MULTIPOLYGON (((-65.50000 -55.20000, -66.45000...",Argentina


In [11]:
statesmerge = states.merge(new_data, how='left', left_on='name', right_on='name')
statesmerge['geometry'] = statesmerge.geometry.simplify(.05)

In [12]:
statesmerge.head()

,geometry,name,cases,deaths
0,"POLYGON ((61.21082 35.65007, 62.23065 35.27066...",Afghanistan,2171.0,64.0
1,"MULTIPOLYGON (((16.32653 -5.87747, 16.86019 -7...",Angola,27.0,2.0
2,"POLYGON ((20.59025 41.85540, 20.46317 41.51509...",Albania,773.0,31.0
3,"POLYGON ((51.57952 24.24550, 51.75744 24.29407...",United Arab Emirates,12481.0,105.0
4,"MULTIPOLYGON (((-65.50000 -55.20000, -66.45000...",Argentina,4415.0,218.0


In [13]:
mymap = folium.Map(location=[0, 0], tiles="OpenStreetMap",zoom_start=2,max_bounds=True,no_wrap=True)
mymap

In [14]:
colormap = branca.colormap.LinearColormap(
    vmin=statesmerge['cases'].min(), 
    vmax=statesmerge['cases'].max(),
    colors=['#FF0000','#FF3300','#ff6600','#ff9900','#FFCC00','#FFFF00','#ccff00','#99ff00','#66ff00','#33ff00','#00FF00'],
    caption="COVID-19 Cases around the world",
)

In [15]:
tooltip = GeoJsonTooltip(
    fields=["name", "cases", "deaths"],
    aliases=["Country:", "COVID-19 Cases:", "COVID-19 Deaths:"],
    localize=True,
    sticky=False,
    labels=True,
    style="""
        background-color: #F0EFEF;
        border: 2px solid red;
        border-radius: 3px;
        box-shadow: 3px;
    """,

)


g = folium.GeoJson(
    statesmerge,
    style_function=lambda x: {
        "fillColor": colormap(x["properties"]["cases"])
        if x["properties"]["cases"] is not None
        else "transparent",
        "color": "black",
        "fillOpacity": 0.4,
    },
    tooltip=tooltip,

).add_to(mymap)

colormap.add_to(mymap)

In [16]:
statesearch = Search(
            layer=g,
            geom_type='Polygon',
            placeholder='Search for Country',
            collapsed=True,
            search_label='name',
            weight=3,
            position='bottomright',
        ).add_to(mymap)

            #mymap.add_child(folium.LatLngPopup())
            #mymap.add_child(folium.ClickForMarker(popup='Waypoint'))
myminimap = MiniMap(toggle_display=True,tile_layer='OpenStreetMap',position='bottomleft',minimized=True)
myminimap.add_to(mymap)


In [17]:
mymap

In [18]:
mymap.save('COVD-19-Cases.html')